<a href="https://colab.research.google.com/github/MeherBhaskar/MNIST-with-Telegram-Feedack/blob/master/MNIST_with_Telegram_feedback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import requests
import keras 
import tensorflow as tf

Using TensorFlow backend.


In [2]:
access_token = '1347289675:AAF2OqATE8Hw7RNyMKXXyk1bxge-GxlGvqQ'

class botCallback(tf.keras.callbacks.Callback):
    def __init__(self,access_token):
        self.access_token = access_token
        self.ping_url = 'https://api.telegram.org/bot'+str(self.access_token)+'/getUpdates'
        self.response = requests.get(self.ping_url).json()
        self.chat_id = self.response['result'][0]['message']['chat']['id']
        self.send_message('Initiating Model Monitoring')

    def send_message(self,message):
        self.ping_url = 'https://api.telegram.org/bot'+str(self.access_token)+'/sendMessage?'+\
                        'chat_id='+str(self.chat_id)+\
                        '&parse_mode=Markdown'+\
                        '&text='+message
        self.response = requests.get(self.ping_url)
    
    def send_photo(self,filepath):
        file_ = open(filepath,'rb')
        file_dict = {'photo':file_}
        self.ping_url = 'https://api.telegram.org/bot'+str(self.access_token)+'/sendPhoto?'+\
                    'chat_id='+str(self.chat_id)
        self.response = requests.post(self.ping_url,files = file_dict)
        file_.close()

    def on_train_begin(self, logs=None):
        message = 'Training started'
        self.send_message(message)

    def on_train_end(self, logs=None):
        message = 'Model Training Completed'
        self.send_message(message)

    def on_train_batch_begin(self, batch, logs=None):
        ##print(logs.keys())
        pass
        
    
    def on_train_batch_end(self, batch, logs=None):
        # # print(logs.keys())  
        # message = '~~~Training Log for Batch {}~~~\n'.format(batch)
        # message += 'Training Accuracy : {:7.2f}%\n Training Loss : {:7.2f}\n'.format(logs['accuracy']*100,logs['loss'])
        # #message += ' Validation Loss : {:7.2f}\n'.format(logs['val_loss'])
        # self.send_message(message)
        pass

    def on_test_batch_begin(self, batch, logs=None):
        ##print(logs.keys())  
        
        pass
    
    def on_test_batch_end(self, batch, logs=None):
        # print(logs.keys())   
        # message = '~~~Test Log for Batch {}~~~\n'.format(batch_size)
        # message += 'Test Accuracy : {:7.2f}%\n Test Loss : {:7.2f}\n'.format(logs['accuracy']*100,logs['loss'])
        
        # #message += ' Validation Loss : {:7.2f}\n'.format(logs['val_loss'])
        # self.send_message(message)
        pass

    def on_epoch_begin(self, epoch, logs=None):
        pass

    def on_epoch_end(self, epoch, logs=None):
        print(logs.keys()) 
        message = '~~~Epoch {} Log~~~\n'.format(epoch)
        
        message += ' Training Accuracy : {:7.2f}%\n Training Loss : {:7.2f}\n'.format(logs['accuracy']*100,logs['loss'])
        message += ' Validation Loss : {:7.2f}\n Validation Accuracy : {:7.2f}%'.format(logs['val_loss'],logs['val_accuracy']*100)
        self.send_message(message)
        pass

In [3]:
bot_callback = botCallback(access_token)

In [4]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [5]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [6]:
print(x_train.shape)
print('Shape after formating', x_train.shape)

x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2],1)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],1)
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)


img_shape = (x_train.shape[1], x_train.shape[2], 1)

x_train/=255
x_test/=255

(60000, 28, 28)
Shape after formating (60000, 28, 28)


In [7]:
from keras.utils import np_utils

num_classes = 10

y_train = keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes=num_classes)

num_pixels = x_train.shape[1]*x_train.shape[2]


In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
from keras.optimizers import SGD

## Build the model

model = Sequential()
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape = img_shape))
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation = 'softmax'))
# compile model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

In [12]:
# Train the model

batch_size = 512
epochs = 1000

checkpoint = tf.keras.callbacks.ModelCheckpoint('/content/sample_data/temp_modek', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, 
                                   verbose=1, mode='min', min_lr=0.00001)
                              
early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=1)

history = model.fit(x_train, y_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (x_test, y_test),
                    callbacks = [ reduce_lr, early_stop, bot_callback])
score = model.evaluate(x_test, y_test, verbose = 1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1000
60000/60000 [==============================] - 5s 88us/step - loss: 0.0208 - accuracy: 0.9934 - val_loss: 0.0281 - val_accuracy: 0.9913
dict_keys(['val_loss', 'val_accuracy', 'loss', 'accuracy', 'lr'])
Epoch 2/1000
60000/60000 [==============================] - 5s 88us/step - loss: 0.0192 - accuracy: 0.9940 - val_loss: 0.0273 - val_accuracy: 0.9916
dict_keys(['val_loss', 'val_accuracy', 'loss', 'accuracy', 'lr'])
Epoch 3/1000
60000/60000 [==============================] - 5s 89us/step - loss: 0.0182 - accuracy: 0.9941 - val_loss: 0.0269 - val_accuracy: 0.9913
dict_keys(['val_loss', 'val_accuracy', 'loss', 'accuracy', 'lr'])
Epoch 4/1000
60000/60000 [==============================] - 5s 89us/step - loss: 0.0176 - accuracy: 0.9941 - val_loss: 0.0275 - val_accuracy: 0.9918

Epoch 00004: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
dict_keys(['val_loss', 'val_accuracy', 'loss', 'accuracy', 'lr'])
10000/1000